In [2]:
from numpy import genfromtxt
from time import time
from datetime import datetime
from dateutil.relativedelta import relativedelta
from sqlalchemy import Column, Integer, Float, Date, String, VARCHAR
from sqlalchemy.ext.declarative import declarative_base
import sqlite3
from sqlalchemy import create_engine
from sqlalchemy.orm import session
import csv
import pandas as pd

In [2]:
file_path = "../Resources/CTA_Ridership_Monthly_Day_Type_Averages.csv"
cta_monthly_df = pd.read_csv(file_path)
cta_monthly_df.head()

,station_id,stationame,month_beginning,avg_weekday_rides,avg_saturday_rides,avg_sunday-holiday_rides,monthtotal
0,40900,Howard,1/1/2001,6233.9,3814.5,2408.6,164447
1,41190,Jarvis,1/1/2001,1489.1,1054.0,718.0,40567
2,40100,Morse,1/1/2001,4412.5,3064.5,2087.8,119772
3,41300,Loyola,1/1/2001,4664.5,3156.0,1952.8,125008
4,40760,Granville,1/1/2001,3109.8,2126.0,1453.8,84189


In [3]:
cta_monthly_df['stationame'] = cta_monthly_df['stationame'].str.replace('/','-')
cta_monthly_df.head()

,station_id,stationame,month_beginning,avg_weekday_rides,avg_saturday_rides,avg_sunday-holiday_rides,monthtotal
0,40900,Howard,1/1/2001,6233.9,3814.5,2408.6,164447
1,41190,Jarvis,1/1/2001,1489.1,1054.0,718.0,40567
2,40100,Morse,1/1/2001,4412.5,3064.5,2087.8,119772
3,41300,Loyola,1/1/2001,4664.5,3156.0,1952.8,125008
4,40760,Granville,1/1/2001,3109.8,2126.0,1453.8,84189


In [4]:
cta_monthly_df.dtypes

station_id                    int64
stationame                   object
month_beginning              object
avg_weekday_rides           float64
avg_saturday_rides          float64
avg_sunday-holiday_rides    float64
monthtotal                    int64
dtype: object

In [5]:
cta_monthly_df['date'] = pd.to_datetime(cta_monthly_df['month_beginning'])
cta_monthly_df.head()

,station_id,stationame,month_beginning,avg_weekday_rides,avg_saturday_rides,avg_sunday-holiday_rides,monthtotal,date
0,40900,Howard,1/1/2001,6233.9,3814.5,2408.6,164447,2001-01-01
1,41190,Jarvis,1/1/2001,1489.1,1054.0,718.0,40567,2001-01-01
2,40100,Morse,1/1/2001,4412.5,3064.5,2087.8,119772,2001-01-01
3,41300,Loyola,1/1/2001,4664.5,3156.0,1952.8,125008,2001-01-01
4,40760,Granville,1/1/2001,3109.8,2126.0,1453.8,84189,2001-01-01


In [6]:
cta_monthly_df.dtypes

station_id                           int64
stationame                          object
month_beginning                     object
avg_weekday_rides                  float64
avg_saturday_rides                 float64
avg_sunday-holiday_rides           float64
monthtotal                           int64
date                        datetime64[ns]
dtype: object

In [7]:
cta_monthly_df_rename = cta_monthly_df.rename(columns={"station_id": "Station_ID", "stationame":"Station_Name"})
cta_monthly_df_rename.head()

,Station_ID,Station_Name,month_beginning,avg_weekday_rides,avg_saturday_rides,avg_sunday-holiday_rides,monthtotal,date
0,40900,Howard,1/1/2001,6233.9,3814.5,2408.6,164447,2001-01-01
1,41190,Jarvis,1/1/2001,1489.1,1054.0,718.0,40567,2001-01-01
2,40100,Morse,1/1/2001,4412.5,3064.5,2087.8,119772,2001-01-01
3,41300,Loyola,1/1/2001,4664.5,3156.0,1952.8,125008,2001-01-01
4,40760,Granville,1/1/2001,3109.8,2126.0,1453.8,84189,2001-01-01


In [8]:
# Reorganizing the columns
cta_monthly_df_reorg = cta_monthly_df_rename[["Station_ID", "Station_Name","date", "avg_weekday_rides", "avg_saturday_rides",
                                "avg_sunday-holiday_rides", "monthtotal"]]
cta_monthly_df_reorg.head()

,Station_ID,Station_Name,date,avg_weekday_rides,avg_saturday_rides,avg_sunday-holiday_rides,monthtotal
0,40900,Howard,2001-01-01,6233.9,3814.5,2408.6,164447
1,41190,Jarvis,2001-01-01,1489.1,1054.0,718.0,40567
2,40100,Morse,2001-01-01,4412.5,3064.5,2087.8,119772
3,41300,Loyola,2001-01-01,4664.5,3156.0,1952.8,125008
4,40760,Granville,2001-01-01,3109.8,2126.0,1453.8,84189


# Extract 10 Year ridership average for each station

In [10]:
max_date = cta_monthly_df_reorg['date'].max()
max_date

Timestamp('2018-12-01 00:00:00')

In [11]:
ten_yrs_ago = max_date - relativedelta(years=11)
ten_yrs_ago

Timestamp('2007-12-01 00:00:00')

In [12]:
cta_monthly_ten_years_data = cta_monthly_df_reorg[cta_monthly_df_reorg['date'] > ten_yrs_ago]

cta_monthly_ten_years_data.head()

,Station_ID,Station_Name,date,avg_weekday_rides,avg_saturday_rides,avg_sunday-holiday_rides,monthtotal
11924,40900,Howard,2008-01-01,5455.5,3572.3,2476.0,146691
11925,41190,Jarvis,2008-01-01,1391.5,994.8,747.8,38330
11926,40100,Morse,2008-01-01,3764.5,2555.8,1892.2,102504
11927,41300,Loyola,2008-01-01,4357.1,2972.3,2116.6,118328
11928,40760,Granville,2008-01-01,3060.5,2277.8,1596.2,84424


In [13]:
cta_monthly_ten_years_data['date'] = pd.to_datetime(cta_monthly_ten_years_data['date']).dt.to_period('y')
cta_monthly_ten_years_data.head()

C:\Users\mdjow\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Station_ID,Station_Name,date,avg_weekday_rides,avg_saturday_rides,avg_sunday-holiday_rides,monthtotal
11924,40900,Howard,2008,5455.5,3572.3,2476.0,146691
11925,41190,Jarvis,2008,1391.5,994.8,747.8,38330
11926,40100,Morse,2008,3764.5,2555.8,1892.2,102504
11927,41300,Loyola,2008,4357.1,2972.3,2116.6,118328
11928,40760,Granville,2008,3060.5,2277.8,1596.2,84424


In [14]:
ten_year_data = cta_monthly_ten_years_data.pivot_table(index=['Station_ID','Station_Name'], 
                    columns='date', 
                    values='monthtotal', 
                    aggfunc='mean')
ten_year_data.head()

,date,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Station_ID,Station_Name,,,,,,,,,,,
40010,Austin-Forest Park,46823.750000,46341.750000,48507.000000,51768.416667,53616.500000,53141.416667,53227.333333,52914.333333,52216.083333,49944.166667,47472.166667
40020,Harlem-Lake,101845.583333,95363.000000,95740.333333,96599.083333,100778.250000,102351.416667,106548.416667,105530.083333,97463.500000,97634.916667,97086.583333
40030,Pulaski-Lake,45024.833333,48082.250000,53863.666667,54576.166667,55984.583333,51132.083333,50526.916667,47168.250000,40027.916667,37620.333333,37237.500000
40040,Quincy-Wells,168745.333333,170798.416667,166625.583333,183112.750000,184427.083333,185044.416667,190696.083333,182758.083333,186506.083333,185204.416667,181934.916667
40050,Davis,105441.000000,103588.500000,102167.500000,103838.750000,104414.833333,103015.666667,104849.916667,103930.083333,100214.166667,97038.833333,96000.416667


In [15]:
ten_year_data = ten_year_data.reset_index().rename_axis(None).rename_axis(None, axis=1)
ten_year_data.head()

,Station_ID,Station_Name,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,40010,Austin-Forest Park,46823.750000,46341.750000,48507.000000,51768.416667,53616.500000,53141.416667,53227.333333,52914.333333,52216.083333,49944.166667,47472.166667
1,40020,Harlem-Lake,101845.583333,95363.000000,95740.333333,96599.083333,100778.250000,102351.416667,106548.416667,105530.083333,97463.500000,97634.916667,97086.583333
2,40030,Pulaski-Lake,45024.833333,48082.250000,53863.666667,54576.166667,55984.583333,51132.083333,50526.916667,47168.250000,40027.916667,37620.333333,37237.500000
3,40040,Quincy-Wells,168745.333333,170798.416667,166625.583333,183112.750000,184427.083333,185044.416667,190696.083333,182758.083333,186506.083333,185204.416667,181934.916667
4,40050,Davis,105441.000000,103588.500000,102167.500000,103838.750000,104414.833333,103015.666667,104849.916667,103930.083333,100214.166667,97038.833333,96000.416667


In [16]:
ten_year_data.count()

Station_ID      146
Station_Name    146
2008            142
2009            142
2010            141
2011            141
2012            143
2013            143
2014            143
2015            144
2016            144
2017            145
2018            145
dtype: int64

In [17]:
ten_year_data['Station_ID'].nunique()

146

In [95]:
engine = create_engine(f'postgresql://postgres:postgres@:5432/CTA')
con = engine.connect()
ten_year_data.to_sql('total_ridership', con)

# Monthly Ridership Data for last one year

In [18]:
one_yrs_ago = max_date - relativedelta(years=1)
one_yrs_ago

Timestamp('2017-12-01 00:00:00')

In [19]:
cta_monthly_one_years_data = cta_monthly_df_reorg[cta_monthly_df_reorg['date'] > one_yrs_ago]

cta_monthly_one_years_data.head()

,Station_ID,Station_Name,date,avg_weekday_rides,avg_saturday_rides,avg_sunday-holiday_rides,monthtotal
29027,40580,54th-Cermak,2018-01-01,1887.9,985.8,605.6,48504
29028,41020,Logan Square,2018-01-01,6877.0,3727.5,2437.6,178391
29029,40800,Sedgwick,2018-01-01,3453.3,1881.0,1243.8,89715
29030,40140,Dempster-Skokie,2018-01-01,1650.7,713.8,478.4,41562
29031,41170,Garfield-Dan Ryan,2018-01-01,2844.7,2193.0,1416.8,78440


In [20]:
# change date to a month format
cta_monthly_one_years_data['date'] = pd.to_datetime(cta_monthly_one_years_data['date']).dt.to_period('m')
cta_monthly_one_years_data.head()

C:\Users\mdjow\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Station_ID,Station_Name,date,avg_weekday_rides,avg_saturday_rides,avg_sunday-holiday_rides,monthtotal
29027,40580,54th-Cermak,2018-01,1887.9,985.8,605.6,48504
29028,41020,Logan Square,2018-01,6877.0,3727.5,2437.6,178391
29029,40800,Sedgwick,2018-01,3453.3,1881.0,1243.8,89715
29030,40140,Dempster-Skokie,2018-01,1650.7,713.8,478.4,41562
29031,41170,Garfield-Dan Ryan,2018-01,2844.7,2193.0,1416.8,78440


In [21]:
one_year_monthly_data = cta_monthly_one_years_data.pivot_table(index=['Station_ID','Station_Name'], 
                    columns='date', 
                    values='monthtotal', 
                    aggfunc='mean')
one_year_monthly_data.head()

,date,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12
Station_ID,Station_Name,,,,,,,,,,,,
40010,Austin-Forest Park,44050.0,42282.0,46618.0,46893.0,48748.0,47426.0,47299.0,50664.0,50229.0,56143.0,47491.0,41823.0
40020,Harlem-Lake,87743.0,82998.0,97050.0,93044.0,100828.0,102085.0,103872.0,110235.0,100340.0,107336.0,92976.0,86532.0
40030,Pulaski-Lake,32537.0,31237.0,35516.0,35442.0,39261.0,37908.0,38226.0,39156.0,37709.0,41243.0,37833.0,40782.0
40040,Quincy-Wells,180738.0,161890.0,186353.0,179467.0,190410.0,186307.0,189258.0,201483.0,177927.0,209691.0,174092.0,145603.0
40050,Davis,89673.0,88892.0,98435.0,97293.0,102257.0,102435.0,103452.0,100324.0,92669.0,105530.0,91374.0,79671.0


In [22]:
one_year_monthly_data = one_year_monthly_data.reset_index().rename_axis(None).rename_axis(None, axis=1)
one_year_monthly_data.head()

,Station_ID,Station_Name,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12
0,40010,Austin-Forest Park,44050.0,42282.0,46618.0,46893.0,48748.0,47426.0,47299.0,50664.0,50229.0,56143.0,47491.0,41823.0
1,40020,Harlem-Lake,87743.0,82998.0,97050.0,93044.0,100828.0,102085.0,103872.0,110235.0,100340.0,107336.0,92976.0,86532.0
2,40030,Pulaski-Lake,32537.0,31237.0,35516.0,35442.0,39261.0,37908.0,38226.0,39156.0,37709.0,41243.0,37833.0,40782.0
3,40040,Quincy-Wells,180738.0,161890.0,186353.0,179467.0,190410.0,186307.0,189258.0,201483.0,177927.0,209691.0,174092.0,145603.0
4,40050,Davis,89673.0,88892.0,98435.0,97293.0,102257.0,102435.0,103452.0,100324.0,92669.0,105530.0,91374.0,79671.0


In [ ]:
# add 1 year monthly ridership data to the database 

one_year_monthly_data.to_sql('one_year_monthly_data', con=engine, if_exists='replace')

# Average Weekday rides for last 10 years

In [23]:
weekdays_ridership = cta_monthly_ten_years_data.pivot_table(index=['Station_ID','Station_Name'], 
                    columns='date', 
                    values='avg_weekday_rides', 
                    aggfunc='mean')
weekdays_ridership.head()

,date,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Station_ID,Station_Name,,,,,,,,,,,
40010,Austin-Forest Park,1879.100000,1859.950000,1926.333333,2053.516667,2119.125000,2102.566667,2106.275000,2107.216667,2080.050000,2017.350000,1934.258333
40020,Harlem-Lake,3954.183333,3668.566667,3683.891667,3705.500000,3854.316667,3921.191667,4061.633333,4027.600000,3812.850000,3754.958333,3746.958333
40030,Pulaski-Lake,1713.191667,1810.875000,2045.058333,2047.841667,2074.391667,1914.666667,1877.808333,1756.533333,1496.033333,1406.025000,1396.133333
40040,Quincy-Wells,7202.466667,7325.200000,7150.058333,7861.691667,8060.041667,7976.741667,8214.075000,7939.733333,8084.891667,8100.841667,7961.400000
40050,Davis,3914.383333,3814.533333,3762.641667,3818.783333,3870.475000,3808.191667,3881.875000,3866.333333,3735.975000,3663.491667,3639.775000


In [24]:
weekdays_ridership = weekdays_ridership.reset_index().rename_axis(None).rename_axis(None, axis=1)
weekdays_ridership.head()

,Station_ID,Station_Name,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,40010,Austin-Forest Park,1879.100000,1859.950000,1926.333333,2053.516667,2119.125000,2102.566667,2106.275000,2107.216667,2080.050000,2017.350000,1934.258333
1,40020,Harlem-Lake,3954.183333,3668.566667,3683.891667,3705.500000,3854.316667,3921.191667,4061.633333,4027.600000,3812.850000,3754.958333,3746.958333
2,40030,Pulaski-Lake,1713.191667,1810.875000,2045.058333,2047.841667,2074.391667,1914.666667,1877.808333,1756.533333,1496.033333,1406.025000,1396.133333
3,40040,Quincy-Wells,7202.466667,7325.200000,7150.058333,7861.691667,8060.041667,7976.741667,8214.075000,7939.733333,8084.891667,8100.841667,7961.400000
4,40050,Davis,3914.383333,3814.533333,3762.641667,3818.783333,3870.475000,3808.191667,3881.875000,3866.333333,3735.975000,3663.491667,3639.775000


In [25]:
weekdays_ridership.to_sql('weekday_data', con)

NameError: name 'con' is not defined

# Average Saturday rides for last 10 years

In [26]:
saturday_ridership = cta_monthly_ten_years_data.pivot_table(index=['Station_ID','Station_Name'], 
                    columns='date', 
                    values='avg_saturday_rides', 
                    aggfunc='mean')
saturday_ridership.head()

,date,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Station_ID,Station_Name,,,,,,,,,,,
40010,Austin-Forest Park,864.433333,864.341667,942.625000,1019.025000,1083.033333,1067.550000,1063.200000,1034.391667,1004.766667,907.825000,802.133333
40020,Harlem-Lake,2340.150000,2298.975000,2287.300000,2329.908333,2481.950000,2503.625000,2638.350000,2612.725000,2103.425000,2308.225000,2255.383333
40030,Pulaski-Lake,1086.333333,1222.550000,1307.533333,1364.058333,1456.850000,1300.541667,1319.208333,1238.633333,1033.108333,961.658333,946.591667
40040,Quincy-Wells,1826.891667,1802.508333,1695.600000,1885.983333,1498.233333,1870.241667,1921.300000,1683.408333,1754.300000,1598.008333,1482.025000
40050,Davis,2976.350000,3015.525000,2995.741667,3033.708333,2953.991667,2923.808333,2953.208333,2828.058333,2706.291667,2516.375000,2436.516667


In [27]:
saturday_ridership = saturday_ridership.reset_index().rename_axis(None).rename_axis(None, axis=1)
saturday_ridership.head()

,Station_ID,Station_Name,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,40010,Austin-Forest Park,864.433333,864.341667,942.625000,1019.025000,1083.033333,1067.550000,1063.200000,1034.391667,1004.766667,907.825000,802.133333
1,40020,Harlem-Lake,2340.150000,2298.975000,2287.300000,2329.908333,2481.950000,2503.625000,2638.350000,2612.725000,2103.425000,2308.225000,2255.383333
2,40030,Pulaski-Lake,1086.333333,1222.550000,1307.533333,1364.058333,1456.850000,1300.541667,1319.208333,1238.633333,1033.108333,961.658333,946.591667
3,40040,Quincy-Wells,1826.891667,1802.508333,1695.600000,1885.983333,1498.233333,1870.241667,1921.300000,1683.408333,1754.300000,1598.008333,1482.025000
4,40050,Davis,2976.350000,3015.525000,2995.741667,3033.708333,2953.991667,2923.808333,2953.208333,2828.058333,2706.291667,2516.375000,2436.516667


In [ ]:
saturday_ridership.to_sql('saturday_data', con)

# Average Sunday/Holiday rides for last 10 years

In [28]:
sunday_holiday_ridership = cta_monthly_ten_years_data.pivot_table(index=['Station_ID','Station_Name'], 
                    columns='date', 
                    values='avg_sunday-holiday_rides', 
                    aggfunc='mean')
sunday_holiday_ridership.head()

,date,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Station_ID,Station_Name,,,,,,,,,,,
40010,Austin-Forest Park,619.066667,640.591667,708.433333,763.350000,789.275000,795.550000,795.150000,756.316667,743.725000,676.033333,599.525000
40020,Harlem-Lake,1517.516667,1542.058333,1540.308333,1597.433333,1649.650000,1687.358333,1810.058333,1781.400000,1498.483333,1661.175000,1582.700000
40030,Pulaski-Lake,776.108333,889.708333,971.891667,1058.066667,1133.000000,995.941667,1010.783333,923.500000,761.733333,754.041667,715.866667
40040,Quincy-Wells,1474.783333,1512.141667,1443.483333,1612.433333,1317.983333,1538.883333,1613.841667,1406.450000,1438.425000,1351.950000,1282.041667
40050,Davis,1875.658333,1964.383333,1904.425000,1972.058333,1910.525000,1961.175000,1986.366667,1970.075000,1840.083333,1750.291667,1680.308333


In [29]:
sunday_holiday_ridership = sunday_holiday_ridership.reset_index().rename_axis(None).rename_axis(None, axis=1)
sunday_holiday_ridership.head()

,Station_ID,Station_Name,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,40010,Austin-Forest Park,619.066667,640.591667,708.433333,763.350000,789.275000,795.550000,795.150000,756.316667,743.725000,676.033333,599.525000
1,40020,Harlem-Lake,1517.516667,1542.058333,1540.308333,1597.433333,1649.650000,1687.358333,1810.058333,1781.400000,1498.483333,1661.175000,1582.700000
2,40030,Pulaski-Lake,776.108333,889.708333,971.891667,1058.066667,1133.000000,995.941667,1010.783333,923.500000,761.733333,754.041667,715.866667
3,40040,Quincy-Wells,1474.783333,1512.141667,1443.483333,1612.433333,1317.983333,1538.883333,1613.841667,1406.450000,1438.425000,1351.950000,1282.041667
4,40050,Davis,1875.658333,1964.383333,1904.425000,1972.058333,1910.525000,1961.175000,1986.366667,1970.075000,1840.083333,1750.291667,1680.308333


In [ ]:
sunday_holiday_ridership.to_sql('sunday_holiday_data', con)

# Add location information

In [3]:
path = "../Resources/CTA_System_Information.csv"
system_info = pd.read_csv(path)
system_info.head()

,STOP_ID,DIRECTION_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,ADA,RED,BLUE,G,...,Pexp,Y,Pnk,O,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards
0,30162,W,18th (54th/Cermak-bound),18th,18th (Pink Line),40830,True,False,False,False,...,False,False,True,False,"(41.857908, -87.669147)",8.0,14920,33.0,343.0,26.0
1,30161,E,18th (Loop-bound),18th,18th (Pink Line),40830,True,False,False,False,...,False,False,True,False,"(41.857908, -87.669147)",8.0,14920,33.0,343.0,26.0
2,30022,N,35th/Archer (Loop-bound),35th/Archer,35th/Archer (Orange Line),40120,True,False,False,False,...,False,False,False,True,"(41.829353, -87.680622)",26.0,14924,56.0,719.0,1.0
3,30023,S,35th/Archer (Midway-bound),35th/Archer,35th/Archer (Orange Line),40120,True,False,False,False,...,False,False,False,True,"(41.829353, -87.680622)",26.0,14924,56.0,719.0,1.0
4,30214,S,35-Bronzeville-IIT (63rd-bound),35th-Bronzeville-IIT,35th-Bronzeville-IIT (Green Line),41120,True,False,False,True,...,False,False,False,False,"(41.831677, -87.625826)",12.0,21194,1.0,25.0,9.0


In [31]:
cta_location = system_info[["MAP_ID", "Location"]]
cta_location.head()

,MAP_ID,Location
0,40830,"(41.857908, -87.669147)"
1,40830,"(41.857908, -87.669147)"
2,40120,"(41.829353, -87.680622)"
3,40120,"(41.829353, -87.680622)"
4,41120,"(41.831677, -87.625826)"


In [32]:
cta_location_rename = cta_location.rename(columns={"MAP_ID":"Station_ID"})
cta_location_rename.head()

,Station_ID,Location
0,40830,"(41.857908, -87.669147)"
1,40830,"(41.857908, -87.669147)"
2,40120,"(41.829353, -87.680622)"
3,40120,"(41.829353, -87.680622)"
4,41120,"(41.831677, -87.625826)"


In [33]:
cta_location_rename.count()

Station_ID    297
Location      297
dtype: int64

In [34]:
cta_location_new = cta_location_rename.drop_duplicates(subset=None, keep="first", inplace=False)
cta_location_new


,Station_ID,Location
0,40830,"(41.857908, -87.669147)"
2,40120,"(41.829353, -87.680622)"
4,41120,"(41.831677, -87.625826)"
6,41270,"(41.816462, -87.619021)"
8,41080,"(41.809209, -87.618826)"
10,41230,"(41.810318, -87.63094)"
12,40130,"(41.80209, -87.618487)"
14,40580,"(41.85177331, -87.75669201)"
16,40910,"(41.780536, -87.630952)"
18,40990,"(41.768367, -87.625724)"


In [35]:
cta_location_new.dtypes

Station_ID     int64
Location      object
dtype: object

,Station_ID,Station_Name,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,40010,Austin-Forest Park,46823.750000,46341.750000,48507.000000,51768.416667,53616.500000,53141.416667,53227.333333,52914.333333,52216.083333,49944.166667,47472.166667
1,40020,Harlem-Lake,101845.583333,95363.000000,95740.333333,96599.083333,100778.250000,102351.416667,106548.416667,105530.083333,97463.500000,97634.916667,97086.583333
2,40030,Pulaski-Lake,45024.833333,48082.250000,53863.666667,54576.166667,55984.583333,51132.083333,50526.916667,47168.250000,40027.916667,37620.333333,37237.500000
3,40040,Quincy-Wells,168745.333333,170798.416667,166625.583333,183112.750000,184427.083333,185044.416667,190696.083333,182758.083333,186506.083333,185204.416667,181934.916667
4,40050,Davis,105441.000000,103588.500000,102167.500000,103838.750000,104414.833333,103015.666667,104849.916667,103930.083333,100214.166667,97038.833333,96000.416667
5,40060,Belmont-O'Hare,117489.500000,113296.416667,120197.500000,129054.916667,138890.083333,142721.833333,141004.916667,148130.250000,149804.333333,144153.583333,125839.000000
6,40070,Jackson-Dearborn,188139.166667,176507.000000,186192.750000,193094.250000,201597.333333,204956.083333,196462.666667,200146.750000,193289.083333,179964.916667,174221.250000
7,40080,Sheridan,133543.500000,131464.166667,136037.500000,142370.416667,147609.000000,149704.416667,151951.916667,153889.666667,155206.666667,143853.000000,132721.166667
8,40090,Damen-Brown,846.333333,47570.000000,54098.666667,58111.833333,63886.000000,66203.250000,67673.416667,69648.250000,74770.750000,72802.083333,71476.750000
9,40100,Morse,109251.416667,113529.250000,114977.500000,123767.083333,110577.083333,126945.166667,137963.916667,136353.166667,133023.166667,126112.250000,122092.916667


In [91]:
cta_ten_year_ridership = pd.merge(ten_year_data, cta_location_new, on="Station_ID")
cta_ten_year_ridership


,Station_ID,Station_Name,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,Location
0,40010,Austin-Forest Park,46823.750000,46341.750000,48507.000000,51768.416667,53616.500000,53141.416667,53227.333333,52914.333333,52216.083333,49944.166667,47472.166667,"(41.870851, -87.776812)"
1,40020,Harlem-Lake,101845.583333,95363.000000,95740.333333,96599.083333,100778.250000,102351.416667,106548.416667,105530.083333,97463.500000,97634.916667,97086.583333,"(41.886848, -87.803176)"
2,40030,Pulaski-Lake,45024.833333,48082.250000,53863.666667,54576.166667,55984.583333,51132.083333,50526.916667,47168.250000,40027.916667,37620.333333,37237.500000,"(41.885412, -87.725404)"
3,40040,Quincy-Wells,168745.333333,170798.416667,166625.583333,183112.750000,184427.083333,185044.416667,190696.083333,182758.083333,186506.083333,185204.416667,181934.916667,"(41.878723, -87.63374)"
4,40050,Davis,105441.000000,103588.500000,102167.500000,103838.750000,104414.833333,103015.666667,104849.916667,103930.083333,100214.166667,97038.833333,96000.416667,"(42.04771, -87.683543)"
5,40060,Belmont-O'Hare,117489.500000,113296.416667,120197.500000,129054.916667,138890.083333,142721.833333,141004.916667,148130.250000,149804.333333,144153.583333,125839.000000,"(41.938132, -87.712359)"
6,40070,Jackson-Dearborn,188139.166667,176507.000000,186192.750000,193094.250000,201597.333333,204956.083333,196462.666667,200146.750000,193289.083333,179964.916667,174221.250000,"(41.878183, -87.629296)"
7,40080,Sheridan,133543.500000,131464.166667,136037.500000,142370.416667,147609.000000,149704.416667,151951.916667,153889.666667,155206.666667,143853.000000,132721.166667,"(41.953775, -87.654929)"
8,40090,Damen-Brown,846.333333,47570.000000,54098.666667,58111.833333,63886.000000,66203.250000,67673.416667,69648.250000,74770.750000,72802.083333,71476.750000,"(41.966286, -87.678639)"
9,40100,Morse,109251.416667,113529.250000,114977.500000,123767.083333,110577.083333,126945.166667,137963.916667,136353.166667,133023.166667,126112.250000,122092.916667,"(42.008362, -87.665909)"


In [57]:
cta_ten_year_ridership.count()

Station_ID      143
Station_Name    143
2008            139
2009            139
2010            139
2011            139
2012            141
2013            141
2014            141
2015            142
2016            142
2017            143
2018            143
Location        143
dtype: int64

In [76]:
# startyear = 2008
# totalyears = 10
# years = [] 
# references = []
# for x in range(0,totalyears+1):
#     if x == 0:
#         year = startyear
#         years.append(year)
#         reference = 2
#         references.append(reference)
#     else:
#         year = year + 1
#         years.append(year)
#         reference = reference + 1
#         references.append(reference)
# year_dict = dict(zip(years, references))
# useryear = 2018
# column = year_dict[useryear]

# cta_ten_year_ridership.iloc[:, column].tolist()

[47472.166666666664,
 97086.58333333333,
 37237.5,
 181934.91666666666,
 96000.41666666667,
 125839.0,
 174221.25,
 132721.16666666666,
 71476.75,
 122092.91666666667,
 76562.25,
 27273.083333333332,
 49792.083333333336,
 27705.166666666668,
 69084.16666666667,
 62399.083333333336,
 44082.5,
 119950.75,
 38823.916666666664,
 41455.5,
 111280.66666666667,
 176745.91666666666,
 52611.666666666664,
 314987.5,
 30963.583333333332,
 52206.25,
 28569.25,
 21102.583333333332,
 93282.91666666667,
 156848.0,
 323870.8333333333,
 88621.33333333333,
 143518.41666666666,
 93572.91666666667,
 335987.5833333333,
 504026.25,
 83567.41666666667,
 23529.666666666668,
 32115.916666666668,
 87961.91666666667,
 36233.916666666664,
 252082.41666666666,
 190704.5,
 56557.916666666664,
 32791.75,
 73658.41666666667,
 29015.75,
 21569.666666666668,
 141491.83333333334,
 171813.41666666666,
 110440.66666666667,
 227219.16666666666,
 143967.16666666666,
 54544.416666666664,
 172655.66666666666,
 12517.583333333

In [92]:
#split location into lat/lon
cta_ten_year_ridership[['lat','lon']] = cta_ten_year_ridership.Location.str.split(expand=True)
cta_ten_year_ridership.head()

,Station_ID,Station_Name,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,Location,lat,lon
0,40010,Austin-Forest Park,46823.750000,46341.750000,48507.000000,51768.416667,53616.500000,53141.416667,53227.333333,52914.333333,52216.083333,49944.166667,47472.166667,"(41.870851, -87.776812)","(41.870851,",-87.776812)
1,40020,Harlem-Lake,101845.583333,95363.000000,95740.333333,96599.083333,100778.250000,102351.416667,106548.416667,105530.083333,97463.500000,97634.916667,97086.583333,"(41.886848, -87.803176)","(41.886848,",-87.803176)
2,40030,Pulaski-Lake,45024.833333,48082.250000,53863.666667,54576.166667,55984.583333,51132.083333,50526.916667,47168.250000,40027.916667,37620.333333,37237.500000,"(41.885412, -87.725404)","(41.885412,",-87.725404)
3,40040,Quincy-Wells,168745.333333,170798.416667,166625.583333,183112.750000,184427.083333,185044.416667,190696.083333,182758.083333,186506.083333,185204.416667,181934.916667,"(41.878723, -87.63374)","(41.878723,",-87.63374)
4,40050,Davis,105441.000000,103588.500000,102167.500000,103838.750000,104414.833333,103015.666667,104849.916667,103930.083333,100214.166667,97038.833333,96000.416667,"(42.04771, -87.683543)","(42.04771,",-87.683543)


In [93]:
#convert lat/lon into numeric
cta_ten_year_ridership['lat'] = cta_ten_year_ridership.lat.str.replace('(' , '')
cta_ten_year_ridership['lat'] = cta_ten_year_ridership.lat.str.replace(',' , '')
cta_ten_year_ridership['lon'] = cta_ten_year_ridership.lon.str.replace(')' , '')
cta_ten_year_ridership['lat'] = pd.to_numeric(cta_ten_year_ridership['lat'])
cta_ten_year_ridership['lon'] = pd.to_numeric(cta_ten_year_ridership['lon'])
cta_ten_year_ridership.head()

,Station_ID,Station_Name,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,Location,lat,lon
0,40010,Austin-Forest Park,46823.750000,46341.750000,48507.000000,51768.416667,53616.500000,53141.416667,53227.333333,52914.333333,52216.083333,49944.166667,47472.166667,"(41.870851, -87.776812)",41.870851,-87.776812
1,40020,Harlem-Lake,101845.583333,95363.000000,95740.333333,96599.083333,100778.250000,102351.416667,106548.416667,105530.083333,97463.500000,97634.916667,97086.583333,"(41.886848, -87.803176)",41.886848,-87.803176
2,40030,Pulaski-Lake,45024.833333,48082.250000,53863.666667,54576.166667,55984.583333,51132.083333,50526.916667,47168.250000,40027.916667,37620.333333,37237.500000,"(41.885412, -87.725404)",41.885412,-87.725404
3,40040,Quincy-Wells,168745.333333,170798.416667,166625.583333,183112.750000,184427.083333,185044.416667,190696.083333,182758.083333,186506.083333,185204.416667,181934.916667,"(41.878723, -87.63374)",41.878723,-87.633740
4,40050,Davis,105441.000000,103588.500000,102167.500000,103838.750000,104414.833333,103015.666667,104849.916667,103930.083333,100214.166667,97038.833333,96000.416667,"(42.04771, -87.683543)",42.047710,-87.683543


In [96]:
#add to sql database
cta_ten_year_ridership.to_sql('ten_year_ridership', con)

# Station Specific Attributes 

In [5]:
# Working with the orignal system_info dataframe, rename columns for conistancy 
system_info.rename(columns = {'STATION_NAME':'Station_Name', 'MAP_ID':'Station_ID', 'RED':'Red', 'BLUE':'Blue', 'G': 'Green', 'BRN': 'Brown',
                'Pexp':'Purple Express', 'Y': 'Yellow', 'Pnk': 'Pink', 'O': 'Orange'}
                 ,inplace = True) 

# system_info.head()

,STOP_ID,DIRECTION_ID,STOP_NAME,Station_Name,STATION_DESCRIPTIVE_NAME,Station_ID,ADA,Red,Blue,Green,...,Purple Express,Yellow,Pink,Orange,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards
0,30162,W,18th (54th/Cermak-bound),18th,18th (Pink Line),40830,True,False,False,False,...,False,False,True,False,"(41.857908, -87.669147)",8.0,14920,33.0,343.0,26.0
1,30161,E,18th (Loop-bound),18th,18th (Pink Line),40830,True,False,False,False,...,False,False,True,False,"(41.857908, -87.669147)",8.0,14920,33.0,343.0,26.0
2,30022,N,35th/Archer (Loop-bound),35th/Archer,35th/Archer (Orange Line),40120,True,False,False,False,...,False,False,False,True,"(41.829353, -87.680622)",26.0,14924,56.0,719.0,1.0
3,30023,S,35th/Archer (Midway-bound),35th/Archer,35th/Archer (Orange Line),40120,True,False,False,False,...,False,False,False,True,"(41.829353, -87.680622)",26.0,14924,56.0,719.0,1.0
4,30214,S,35-Bronzeville-IIT (63rd-bound),35th-Bronzeville-IIT,35th-Bronzeville-IIT (Green Line),41120,True,False,False,True,...,False,False,False,False,"(41.831677, -87.625826)",12.0,21194,1.0,25.0,9.0


In [6]:
# create new dataframe with system_info, drop columns were are not using, drop duplicates, move Station_ID to the first Column
system_df = system_info 
system_df.drop(['STOP_ID','DIRECTION_ID', 'STOP_NAME', 'STATION_DESCRIPTIVE_NAME', 'Location'], axis = 1, inplace=True)
system_df.set_index('Station_ID', inplace = True)
system_df.reset_index(inplace=True)
system_df.drop_duplicates(subset='Station_Name', keep= 'first', inplace =True)
system_df.drop_duplicates(subset='Station_ID', keep= 'first', inplace =True)
# system_df.head()

,Station_ID,Station_Name,ADA,Red,Blue,Green,Brown,P,Purple Express,Yellow,Pink,Orange,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards
0,40830,18th,True,False,False,False,False,False,False,False,True,False,8.0,14920,33.0,343.0,26.0
2,40120,35th/Archer,True,False,False,False,False,False,False,False,False,True,26.0,14924,56.0,719.0,1.0
4,41120,35th-Bronzeville-IIT,True,False,False,True,False,False,False,False,False,False,12.0,21194,1.0,25.0,9.0
6,41270,43rd,True,False,False,True,False,False,False,False,False,False,12.0,4301,4.0,162.0,9.0
8,41080,47th,True,False,False,True,False,False,False,False,False,False,12.0,21192,4.0,448.0,9.0


In [ ]:
#add to sql database
system_df.to_sql('system', con)